In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder
from joblib import Parallel, delayed
import os
import pickle
import warnings

warnings.filterwarnings('ignore')

# --- CONFIGURATION ---
CONF = {
    'input_file': '/content/sales_data_final.csv', # Update path
    'forecast_horizon_weeks': 104, # Predict next 2 years
    'prophet_model_dir': 'models/prophet_individual',
    'global_model_dir': 'models/global_lgbm',
    'n_jobs': -1 # Use all cores
}

for d in [CONF['prophet_model_dir'], CONF['global_model_dir']]:
    os.makedirs(d, exist_ok=True)

# --- 1. DATA PREPROCESSING ---
def load_and_clean_data(filepath):
    print(">>> Loading Data...")
    df = pd.read_csv(filepath)

    # Identify date columns (assuming format YYYY-MM-DD or similar in cols)
    id_vars = ['Product_Code'] # Add Category if exists
    date_cols = [c for c in df.columns if c not in id_vars]

    # Melt to Long Format
    df_long = df.melt(id_vars=id_vars, value_vars=date_cols, var_name='ds', value_name='y')
    df_long['ds'] = pd.to_datetime(df_long['ds'])

    # Handle Negative/Zero Sales with Log Transform
    # log1p(x) = log(x + 1). This ensures 0 sales -> 0, and no negatives.
    df_long['y_log'] = np.log1p(df_long['y'])

    df_long = df_long.sort_values(['Product_Code', 'ds']).reset_index(drop=True)
    print(f"Data Loaded: {df_long.shape[0]} rows, {df_long['Product_Code'].nunique()} products.")
    return df_long

df_clean = load_and_clean_data(CONF['input_file'])
print(df_clean)
df_clean.to_csv('./cleaned_dataset_for_training.csv')

>>> Loading Data...
Data Loaded: 42172 rows, 811 products.


In [ ]:
# --- 2. PROPHET TRAINING & BASELINE GENERATION ---

def train_prophet_single(group, product_id, horizon):
    """
    Trains Prophet on log-sales.
    Returns: DataFrame containing (ds, yhat_log, Product_Code, type='history'|'future')
    """
    try:
        # Prepare data for Prophet
        df_p = group[['ds', 'y_log']].rename(columns={'y_log': 'y'})

        # Force yearly seasonality because we only have 52 weeks (risky but necessary)
        m = Prophet(yearly_seasonality=True,
                    weekly_seasonality=False,
                    daily_seasonality=False)

        # Add holiday effect if relevant (optional)
        # m.add_country_holidays(country_name='US')

        m.fit(df_p)

        # Save model (optional, uses disk space)
        with open(f"{CONF['prophet_model_dir']}/{product_id}.pkl", 'wb') as f:
            pickle.dump(m, f)

        # Create Future Dataframe (History + Future)
        future = m.make_future_dataframe(periods=horizon, freq='W')
        forecast = m.predict(future)

        # Clean up result
        forecast = forecast[['ds', 'yhat']].rename(columns={'yhat': 'prophet_pred_log'})
        forecast['Product_Code'] = product_id

        # Mark rows as training or future for easy splitting later
        max_train_date = df_p['ds'].max()
        forecast['split_type'] = forecast['ds'].apply(
            lambda x: 'train' if x <= max_train_date else 'future'
        )

        return forecast
    except Exception as e:
        print(f"Error on {product_id}: {e}")
        return pd.DataFrame()

print(f">>> Starting Prophet Training for {df_clean['Product_Code'].nunique()} products...")
print(f"    Forecasting Horizon: {CONF['forecast_horizon_weeks']} weeks")

products = df_clean['Product_Code'].unique()
results = Parallel(n_jobs=CONF['n_jobs'], verbose=5)(
    delayed(train_prophet_single)(
        df_clean[df_clean['Product_Code'] == p],
        p,
        CONF['forecast_horizon_weeks']
    ) for p in products
)

# Combine all Prophet outputs
df_prophet_full = pd.concat(results)

# Save intermediate file (Crucial for Phase 3 Inference)
df_prophet_full.to_pickle(f"{CONF['global_model_dir']}/prophet_full_forecast.pkl")
print(">>> Phase 1 Complete. Prophet Baseline Generated.")

>>> Starting Prophet Training for 811 products...
    Forecasting Horizon: 104 weeks


KeyboardInterrupt: 

In [ ]:
# --- 3. FEATURE ENGINEERING ---

def create_features(df):
    df = df.copy()

    # 1. Time Features
    df['month'] = df['ds'].dt.month
    df['week'] = df['ds'].dt.isocalendar().week.astype(int)
    df['year'] = df['ds'].dt.year

    # 2. Cyclical Encoding (Crucial for Seasonality)
    # Encodes that Month 12 is close to Month 1
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['week_sin'] = np.sin(2 * np.pi * df['week'] / 52)
    df['week_cos'] = np.cos(2 * np.pi * df['week'] / 52)

    return df

print(">>> Preparing LightGBM Dataset...")

# Merge Prophet Predictions with Actuals for Training
# We only want the 'train' rows from Prophet for the LightGBM training set
df_train_prophet = df_prophet_full[df_prophet_full['split_type'] == 'train']

df_ensemble = pd.merge(df_clean, df_train_prophet[['ds', 'Product_Code', 'prophet_pred_log']],
                       on=['ds', 'Product_Code'], how='inner')

# Apply Feature Engineering
df_ensemble = create_features(df_ensemble)

# Encode Product Code
le = LabelEncoder()
df_ensemble['Product_Code_Encoded'] = le.fit_transform(df_ensemble['Product_Code'])

# Save Encoder
with open(f"{CONF['global_model_dir']}/product_encoder.pkl", 'wb') as f:
    pickle.dump(le, f)

>>> Preparing LightGBM Dataset...


In [ ]:
# --- 4. GLOBAL LIGHTGBM TRAINING ---

print(">>> Training Global LightGBM Model...")

features = [
    'prophet_pred_log',
    'Product_Code_Encoded',
    'month_sin', 'month_cos',
    'week_sin', 'week_cos',
    'year'
]
target = 'y_log' # Train on Log Sales

# Robust Time Split
# We sort by date. Last 4 weeks = Validation, Rest = Train.
unique_dates = sorted(df_ensemble['ds'].unique())
split_date = unique_dates[-4] # Reserve last 4 weeks for validation

print(f"    Splitting Train/Valid at: {split_date}")

train_mask = df_ensemble['ds'] <= split_date
valid_mask = df_ensemble['ds'] > split_date

X_train = df_ensemble.loc[train_mask, features]
y_train = df_ensemble.loc[train_mask, target]
X_valid = df_ensemble.loc[valid_mask, features]
y_valid = df_ensemble.loc[valid_mask, target]

# Create Datasets
dtrain = lgb.Dataset(X_train, label=y_train)
dvalid = lgb.Dataset(X_valid, label=y_valid, reference=dtrain)

# Hyperparameters
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.01, # Lower learning rate for better generalization on small data
    'num_leaves': 31,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'seed': 42
}

model_lgb = lgb.train(
    params,
    dtrain,
    num_boost_round=2000,
    valid_sets=[dtrain, dvalid],
    valid_names=['train', 'valid'],
    callbacks=[
        lgb.early_stopping(stopping_rounds=100),
        lgb.log_evaluation(period=200)
    ]
)

# Save LightGBM
model_lgb.save_model(f"{CONF['global_model_dir']}/lgb_model.txt")
print(">>> LightGBM Model Trained and Saved.")

>>> Training Global LightGBM Model...
    Splitting Train/Valid at: 2025-12-09 00:00:00
Training until validation scores don't improve for 100 rounds
[200]	train's rmse: 0.35173	valid's rmse: 0.326733
[400]	train's rmse: 0.276265	valid's rmse: 0.253061
[600]	train's rmse: 0.269306	valid's rmse: 0.245456
[800]	train's rmse: 0.266424	valid's rmse: 0.243371
[1000]	train's rmse: 0.264489	valid's rmse: 0.242897
Early stopping, best iteration is:
[1075]	train's rmse: 0.263854	valid's rmse: 0.242791
>>> LightGBM Model Trained and Saved.


In [ ]:
# --- 5. INFERENCE & FORECAST GENERATION ---

print(">>> Generating Final Forecasts...")

# 1. Get the Future Prophet Data (calculated in Phase 1)
df_future = df_prophet_full[df_prophet_full['split_type'] == 'future'].copy()

if df_future.empty:
    raise ValueError("No future dates found in Prophet predictions. Check Phase 1 horizon.")

# 2. Feature Engineering (Must be identical to Training)
df_future = create_features(df_future)

# 3. Encoding
# Use the safe transform (handle potential new products gracefully, though unlikely here)
df_future['Product_Code_Encoded'] = df_future['Product_Code'].apply(
    lambda x: le.transform([x])[0] if x in le.classes_ else -1
)

# Filter out unknown products (if any)
df_future = df_future[df_future['Product_Code_Encoded'] != -1]

# 4. LightGBM Prediction
print(f"    Predicting for {df_future.shape[0]} future rows...")
# Predict Log Sales
df_future['lgb_pred_log'] = model_lgb.predict(df_future[features])

# 5. Inverse Transformation (Log -> Normal Sales)
# We clip at 0 to ensure no negatives, though expm1 usually handles this
df_future['final_predicted_sales'] = np.expm1(df_future['lgb_pred_log']).clip(lower=0)

# 6. Format Final Output
final_submission = df_future[['ds', 'Product_Code', 'final_predicted_sales']].copy()
final_submission = final_submission.sort_values(['Product_Code', 'ds'])

output_path = 'final_sales_forecast_2years.csv'
final_submission.to_csv(output_path, index=False)

print(f"✅ Pipeline Finished! Forecasts saved to: {output_path}")
print(final_submission.head())

>>> Generating Final Forecasts...
    Predicting for 84344 future rows...
✅ Pipeline Finished! Forecasts saved to: final_sales_forecast_2years.csv
           ds Product_Code  final_predicted_sales
52 2026-01-04           P1              12.903823
53 2026-01-11           P1              13.124837
54 2026-01-18           P1              10.717070
55 2026-01-25           P1               9.688339
56 2026-02-01           P1              11.157923


In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from joblib import Parallel, delayed
from sqlalchemy import create_engine
import os
import pickle
import warnings

warnings.filterwarnings('ignore')

# --- CONFIGURATION ---
CONF = {
    'input_file': '/content/sales_data_final.csv',
    'forecast_horizon_weeks': 104,  # 2 Years
    'prophet_model_dir': 'models/prophet_individual',
    'global_model_dir': 'models/global_lgbm',
    'n_jobs': -1
}

for d in [CONF['prophet_model_dir'], CONF['global_model_dir']]:
    os.makedirs(d, exist_ok=True)

# --- 1. DATA PREPROCESSING ---
def load_and_clean_data(filepath):
    print(">>> Phase 1: Loading & Preprocessing...")
    df = pd.read_csv(filepath)
    id_vars = ['Product_Code']
    date_cols = [c for c in df.columns if c not in id_vars]

    df_long = df.melt(id_vars=id_vars, value_vars=date_cols, var_name='ds', value_name='y')
    df_long['ds'] = pd.to_datetime(df_long['ds'])

    # Log Transform for Training (Handle 0s and skew)
    df_long['y_log'] = np.log1p(df_long['y'])

    df_long = df_long.sort_values(['Product_Code', 'ds']).reset_index(drop=True)
    return df_long

df_clean = load_and_clean_data(CONF['input_file'])

# --- 2. PROPHET FORECAST GENERATION (HISTORY + FUTURE) ---
def train_prophet_single(group, product_id, horizon):
    try:
        df_p = group[['ds', 'y_log']].rename(columns={'y_log': 'y'})

        # Hardcoded seasonality for 52-week data
        m = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
        m.fit(df_p)

        future = m.make_future_dataframe(periods=horizon, freq='W')
        forecast = m.predict(future)

        forecast = forecast[['ds', 'yhat']].rename(columns={'yhat': 'prophet_pred_log'})
        forecast['Product_Code'] = product_id

        max_train_date = df_p['ds'].max()
        forecast['split_type'] = forecast['ds'].apply(lambda x: 'train' if x <= max_train_date else 'future')

        return forecast
    except Exception as e:
        return pd.DataFrame()

print(f">>> Phase 2: Generating Prophet Baselines (Horizon: {CONF['forecast_horizon_weeks']} weeks)...")
results = Parallel(n_jobs=CONF['n_jobs'], verbose=0)(
    delayed(train_prophet_single)(df_clean[df_clean['Product_Code'] == p], p, CONF['forecast_horizon_weeks'])
    for p in df_clean['Product_Code'].unique()
)
df_prophet_full = pd.concat(results)

# --- 3. FEATURE ENGINEERING ---
def create_features(df):
    df = df.copy()
    df['month'] = df['ds'].dt.month
    df['week'] = df['ds'].dt.isocalendar().week.astype(int)
    df['year'] = df['ds'].dt.year
    # Cyclical Features
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['week_sin'] = np.sin(2 * np.pi * df['week'] / 52)
    df['week_cos'] = np.cos(2 * np.pi * df['week'] / 52)
    return df

# Merge & Prepare
df_train_prophet = df_prophet_full[df_prophet_full['split_type'] == 'train']
df_ensemble = pd.merge(df_clean, df_train_prophet[['ds', 'Product_Code', 'prophet_pred_log']],
                       on=['ds', 'Product_Code'], how='inner')
df_ensemble = create_features(df_ensemble)

le = LabelEncoder()
df_ensemble['Product_Code_Encoded'] = le.fit_transform(df_ensemble['Product_Code'])

# --- 4. LIGHTGBM TRAINING & SCORECARD ---
print(">>> Phase 3: Training Ensemble & Generating Scorecard...")

features = ['prophet_pred_log', 'Product_Code_Encoded', 'month_sin', 'month_cos', 'week_sin', 'week_cos', 'year']
target = 'y_log'

# Time-based Split (Last 4 weeks for Validation)
unique_dates = sorted(df_ensemble['ds'].unique())
split_date = unique_dates[-4]

train_mask = df_ensemble['ds'] <= split_date
valid_mask = df_ensemble['ds'] > split_date

X_train, y_train = df_ensemble.loc[train_mask, features], df_ensemble.loc[train_mask, target]
X_valid, y_valid = df_ensemble.loc[valid_mask, features], df_ensemble.loc[valid_mask, target]

# Train
model_lgb = lgb.train(
    {'objective': 'regression', 'metric': 'rmse', 'learning_rate': 0.02, 'verbose': -1},
    lgb.Dataset(X_train, label=y_train),
    num_boost_round=1000,
    valid_sets=[lgb.Dataset(X_valid, label=y_valid)],
    callbacks=[lgb.early_stopping(50)]
)

# --- SCORECARD GENERATION ---
# Predict on Validation Set
preds_log = model_lgb.predict(X_valid)
preds_real = np.expm1(preds_log).clip(min=0) # Convert back to normal scale
actuals_real = np.expm1(y_valid).values

# Calculate Metrics
mae = mean_absolute_error(actuals_real, preds_real)
mse = mean_squared_error(actuals_real, preds_real)
rmse = np.sqrt(mse)
r2 = r2_score(actuals_real, preds_real)

# WMAPE Calculation (Sum of Absolute Errors / Sum of Actuals)
wmape = np.sum(np.abs(actuals_real - preds_real)) / np.sum(actuals_real)
accuracy = 1.0 - wmape

print("\n" + "="*40)
print("       MODEL PERFORMANCE SCORECARD       ")
print("="*40)
print(f" Dataset Split Date : {split_date}")
print("-" * 40)
print(f" R-Squared (R²)     : {r2:.4f}  (Fit Quality)")
print(f" MAE                : {mae:.2f}  (Avg Error per unit)")
print(f" RMSE (MMSE Proxy)  : {rmse:.2f}  (Penalizes large errors)")
print(f" WMAPE              : {wmape:.2%}")
print("-" * 40)
print(f" >> ACCURACY        : {accuracy:.2%} <<")
print("="*40 + "\n")

>>> Phase 1: Loading & Preprocessing...
>>> Phase 2: Generating Prophet Baselines (Horizon: 104 weeks)...
>>> Phase 3: Training Ensemble & Generating Scorecard...
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[315]	valid_0's rmse: 0.239147

       MODEL PERFORMANCE SCORECARD       
 Dataset Split Date : 2025-12-09 00:00:00
----------------------------------------
 R-Squared (R²)     : 0.9540  (Fit Quality)
 MAE                : 1.34  (Avg Error per unit)
 RMSE (MMSE Proxy)  : 2.22  (Penalizes large errors)
 WMAPE              : 15.11%
----------------------------------------
 >> ACCURACY        : 84.89% <<

>>> Phase 4: Production Forecast & Database Export...


TypeError: clip() got an unexpected keyword argument 'min'

In [ ]:

# --- 5. FINAL FORECAST & DB EXPORT ---
print(">>> Phase 4: Production Forecast & Database Export...")

# Prepare Future Data
df_future = df_prophet_full[df_prophet_full['split_type'] == 'future'].copy()
df_future = create_features(df_future)
df_future['Product_Code_Encoded'] = df_future['Product_Code'].apply(
    lambda x: le.transform([x])[0] if x in le.classes_ else -1
)
df_future = df_future[df_future['Product_Code_Encoded'] != -1]

# Predict
df_future['predicted_log'] = model_lgb.predict(df_future[features])
df_future['predicted_sales'] = np.expm1(df_future['predicted_log']).clip(lower=0)

# Format for DB (Clean Columns)
df_export = df_future[['ds', 'Product_Code', 'predicted_sales']].rename(
    columns={'ds': 'forecast_date', 'Product_Code': 'product_code'}
)

# Also save CSV as backup
df_export.to_csv('final_forecast_backup.csv', index=False)

>>> Phase 4: Production Forecast & Database Export...
❌ Database Error: (psycopg2.OperationalError) could not translate host name "1912@db.vrxpfivbtfzsltxqlbwu.supabase.co" to address: Name or service not known

(Background on this error at: https://sqlalche.me/e/20/e3q8)
   (Check your connection string in CONF)


In [ ]:
model_lgb.save_model('./trained_lgb_model_for_metrics.txt')

In [ ]:
df_ensemble.to_csv('./latest_df_ensemble.csv')